In [1]:
# Import seaborn and apply its plotting styles
import seaborn as sns
sns.set(font_scale=2, style="white")

# import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.patches as mpatches
# set plotting size parameter
plt.rcParams['figure.figsize'] = (17, 7)

# import pandas & numpy library
import pandas as pd
import numpy as np

# Statmodels & patsy
import patsy
import statsmodels.api as sm
from scipy import stats

import re
import csv

# **Clean Data**

Drop/rename and create columns

In [2]:
data = pd.read_csv('164_data.csv')

# Drop and rename columns
data = data.drop(labels = ['EndDate', 'SC0', 'SC3', 'Duration (in seconds)', 'Status', 'IPAddress', 'Progress', 
        'Finished', 'RecordedDate', 'ResponseId',
        'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
        'ExternalReference', 'LocationLatitude', 'LocationLongitude',
        'DistributionChannel', 'UserLanguage', 'Q9',
        'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q20',
        'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29',
        'Q32_1', 'Q32_2', 'Q32_3',
        'Q32_4', 'Q32_5', 'Q32_6', 'Q32_7', 'Q32_8', 'Q32_9', 'Q32_10',
        'Q32_11', 'Q32_12', 'Q32_13', 'Q32_14', 'Q32_15', 'Q32_16'], axis=1)

data.columns = ['StartDate', 'Q_Eat', 'Q_Sleep', 'Q_Comfort', 'Q_Exercise',
       'C_First Click', 'C_Last Click', 'C_Page Submit',
       'C_Click Count', 'E_First Click', 'E_Last Click',
       'E_Page Submit', 'E_Click Count',
       'Q_Age', 'Q_Gender', 'Q_Ethn', 'Q_Income', 'Q_FinSec', 'PSS',
       'Worry', 'IM', 'IR', 'ER', 'AM', 'KeyResponses', 'PlayerScore']

data = data.iloc[1:]

# Add column describing experimental/control group
data = data.assign(Group= data['C_Page Submit'].isna().apply(lambda x: 'Experimental' if x == True else 'Control'))

In [3]:
# Combine column repeats
def combine_CE(data):
    CE_cols = np.unique([col[1] for col in data.columns.str.split('_') if col[0] in ['C', 'E']])
    for label in CE_cols:
        C_col = f"C_{label}"
        E_col = f"E_{label}"
        if C_col in data.columns and E_col in data.columns:
            data = data.assign(**{label: data[C_col].fillna(data[E_col])})
            data = data.drop(labels= [C_col, E_col], axis=1)

    return data

data = combine_CE(data)

Extract demographic information since it's not being included in any analysis

In [4]:
# Split data into data for analysis and demographic information
demographic_info = data[['Q_Age', 'Q_Gender', 'Q_Ethn', 'Q_Income', 'Q_FinSec']]
data = data[data.columns.difference(['Q_Age', 'Q_Gender', 'Q_Ethn', 'Q_Income', 'Q_FinSec'])].drop(labels= ['First Click', 'Last Click'], axis=1)

Covert numeric columns to floats, add SDI column, and reorder columns/add multi-index

In [5]:
# Convert numeric columns to floats for handling
def float_convert(value):
    try:
        return float(value)
    except:
        return np.nan

num_cols = ['PSS', 'Worry', 'IM', 'IR', 'ER', 'AM', 'PlayerScore', 'Page Submit']
for col in num_cols:
    data[col] = data[col].apply(float_convert)

In [6]:
# Add column for self-determination scale based on the formula
data = data.assign(SDI= 2*data["IM"]+data["IR"]-data["ER"]-2*data["AM"])

In [7]:
# Reorder columns for salience and create multi-index
data = data[['StartDate', 'Group', 'Page Submit', 'PlayerScore', 'Worry', 'PSS', 'SDI', 'IM', 'IR', 'ER', 'AM',
       'Q_Comfort', 'Q_Eat', 'Q_Exercise', 'Q_Sleep', 'Click Count', 'KeyResponses']]

multi_index = [
    ('Metadata', 'StartDate'),
    ('Metadata', 'Group'),
    ('Task', 'Page Submit'),
    ('Task', 'PlayerScore'),
    ('Psych', 'Worry'),
    ('Psych', 'PSS'),
    ('SIMS', 'SDI'),
    ('SIMS', 'IM'),
    ('SIMS', 'IR'),
    ('SIMS', 'ER'),
    ('SIMS', 'AM'),
    ('Questions', 'Comfort'),
    ('Questions', 'Eat'),
    ('Questions', 'Exercise'),
    ('Questions', 'Sleep'),
    ('Interactions', 'Click Count'),
    ('Interactions', 'KeyResponses')
]

# Assign MultiIndex to DataFrame columns
data.columns = pd.MultiIndex.from_tuples(multi_index)

In [8]:
# Remove rows will null values
data = data[data[['Task', 'Psych', 'SIMS']].isna().any(axis=1) == False]

Convert body state information to make it more readable

In [9]:
# Function for coverting comfort column into numeric values (Scale: -2 --> 2)
def convert_comfort(input):
    output = 0
    if "Very" in input:
        output = 2
    elif "Somewhat" in input:
        output = 1
    else:
        output = 0
    
    if "uncomfortable" in input:
        output *= -1
    
    return output

In [10]:
def convert_hrs(input):
    output = input.replace('hours', 'hrs')
    output = output.replace('< ', '0-')
    output = output.replace('≤ ', '0-')
    output = output.replace('≥ ', '')
    if '8 ' in output:
        output = output.replace('8', '8+')
    

In [14]:
# Convert all columns in Questions to more readable format
data['Questions'] = data['Questions'].assign(Comfort= data['Questions', 'Comfort'].map(convert_comfort), 
    Eat= data['Questions', 'Eat'].map(convert_hrs), Sleep= data['Questions', 'Sleep'].map(convert_hrs))
data['Questions', 'Exercise'] = data['Questions', 'Exercise'].apply(lambda str: str.replace('days per week', 'd/wk'))

TypeError: argument of type 'int' is not iterable

In [ ]:
data['Questions', 'Sleep'].unique()

In [ ]:
data['Questions', 'Eat'].unique()

In [ ]:
# Data after cleaning
data.head()

**Separate data into two data frames:** Control and Experimental
- *(makes some functions easier, but these dataframes will not always be used)*

In [ ]:
# Create two dataframes: separate data from each group
E_data = data[data['Metadata','Group'] == 'Experimental']
C_data = data[data['Metadata','Group'] == 'Control']

# **Plotting**

### Distributions of numeric variables

*For measures that are taken before the task commences (PSS and Worry), these distributions should look relatively simiar between groups.
All other distributions might be affected by group*

In [ ]:
# Plot distributions of all numeric variables between groups
def btwn_groups_plots(data, subplots, fig_size):
    v, h = subplots[:2]
    fig, axes = plt.subplots(v, h, figsize=fig_size)
    label_size =12
    i = 0

    if v * h == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    # Use second level of multi-index to index columns for plotting
    data2 = data.droplevel(level=0, axis=1)

    # Only plot numeric columns
    for col in data[['Task', 'Psych', 'SIMS']].columns.get_level_values(1):
        sns.histplot(data= data2, x= col, ax=axes[i], hue= 'Group', kde= True, common_norm= True)
        axes[i].set_title(col, fontsize=label_size)
        axes[i].set_xlabel(col, fontsize=label_size)
        axes[i].set_ylabel('Density', fontsize=label_size)
        axes[i].tick_params(axis="both", labelsize=label_size)
        i += 1

    fig.delaxes(axes[-1])
    plt.tight_layout()

btwn_groups_plots(data, [len(num_cols)//2+1, 2, 1], [15, 25])

## Scatterplots of numeric variables vs. PlayerScore and Page Submit (quitting time)

In [ ]:
# Functions relationships of all numeric variable to task score and quitting time
def within_groups_plots(data, subplots, fig_size):
    v, h = subplots[:2]
    fig, axes = plt.subplots(v, h, figsize=fig_size)
    label_size = 9
    i = 0

    if v * h == 1:
        axes = [axes]  # Convert single-axis to a list
    else:
        axes = axes.flatten()

    data2 = data.droplevel(level=0, axis=1)
    for y_col in data['Task'].columns:
        for x_col in data[['Psych', 'SIMS']].columns.get_level_values(1):
            sns.scatterplot(data= data2, x= x_col, y= y_col, hue= "Group", ax=axes[i])
            axes[i].set_title(f"{y_col} vs. {x_col}", fontsize=label_size)
            axes[i].set_xlabel(x_col, fontsize=label_size)
            axes[i].set_ylabel(y_col, fontsize=label_size)
            axes[i].tick_params(axis="both", labelsize=label_size)
            axes[i].legend(fontsize= label_size)
            i += 1

    if len(subplots) == 3:
        extra_axes = v * h - i
        for j in range(extra_axes):
            plt.delaxes(axes[i + j])
            
    plt.tight_layout()

In [ ]:
# Plot experimental group data
within_groups_plots(data, [4, 4, 1], [15, 12])

## Score/Quitting Time Distribution Boxplots

In [ ]:
# Plot distributions of player score between groups
plt.figure(figsize= (15,5))
sns.boxplot(data=data.droplevel(level=0, axis=1), x="PlayerScore", hue="Group")
plt.title("Distribution of PlayerScore")
plt.legend(loc= "upper right")

In [ ]:
# Plot distributions of quitting time between groups
plt.figure(figsize= (15,5))
sns.boxplot(data=data.droplevel(level=0, axis=1), x="Page Submit", hue="Group")
plt.title("Distribution of Submission Time")
plt.legend(loc= "upper right")

# **Spearman's Coefficients and Correlations**

Spearman's Rank Correlation Coefficients: describe the correlation of numeric variables to score and quitting time

In [ ]:
# Calculate spearman correlations between variables
spearman_E = E_data.droplevel(level=0, axis=1).corr(method= 'spearman').iloc[2:, :2]
spearman_C = C_data.droplevel(level=0, axis=1).corr(method= 'spearman').iloc[2:, :2]

Experimental group correlations

In [ ]:
# Spearman's correlations for the experiemental group
spearman_E

Control group correlations

In [ ]:
# Spearman's correlations for the control group
spearman_C

## Plot correlation coefficients to visualize which variables are most correlated to score/quitting time

Correlation plot for quitting time

In [ ]:
# Plot correlations to quitting time of all survey results
C_bar = sns.barplot(data= spearman_C["Page Submit"], color= 'red')
C_patch = mpatches.Patch(color='red', label='Control')
E_bar = sns.barplot(data= spearman_E["Page Submit"], color= 'blue')
E_patch = mpatches.Patch(color='blue', label='Experimental')
plt.legend(handles= [C_patch, E_patch])
plt.title("Spearman's Correlations to Page Submit")
plt.ylabel("Correlation Coefficient")
plt.xlabel("Survey Score")

Correlation plot for score

In [ ]:
# Plot correlations to score of all survey results
C_bar = sns.barplot(data= spearman_C["PlayerScore"], color= 'red')
C_patch = mpatches.Patch(color='red', label='Control')
E_bar = sns.barplot(data= spearman_E["PlayerScore"], color= 'blue')
E_patch = mpatches.Patch(color='blue', label='Experimental')
plt.legend(handles= [C_patch, E_patch])
plt.title("Spearman's Correlations to PlayerScore")
plt.ylabel("Correlation Coefficient")
plt.xlabel("Survey Score")

## P-values for correlations

In [ ]:
# Create function to get p-values
def get_pval(data, degrees_of_freedom, tails= "two-sided"):
    tstat_df = data.apply(lambda r: r * np.sqrt(len(C_data) - 2) / np.sqrt(1 - r**2))
    if tails == "one-sided":
        pval_df = tstat_df.apply(lambda t: stats.t.cdf(t, degrees_of_freedom))
    elif tails == "two-sided":
        pval_df = tstat_df.apply(lambda t: 2 * (1 - stats.t.cdf(abs(t), degrees_of_freedom)))
    else:
        raise ValueError("tails must be defined")
    
    return pval_df

In [ ]:
# Call get_pval function to create dataframes for the experiemental (E) and control (C) groups with significance of correlations
C_pvalues_df = get_pval(spearman_C, len(C_data)-1)
E_pvalues_df = get_pval(spearman_E, len(E_data)-1)

p-values for the control group

In [ ]:
# Look at p-values for control
C_pvalues_df

p-values for the experimental group

In [ ]:
# Look at p-values for experimental
E_pvalues_df

**Which p-values are significant for each group?**

In [ ]:
# Function for printing which p-values are significant
def significant_pval(data, group):
    output = f"{group}: Significant P-values\n\n"
    threshold = 0.05
    sig_data = data.apply(lambda p: p <= threshold)
    for row in data.index:
        for col in data.columns:
            if sig_data.loc[row, col] == True:
                output += f"{row} vs. {col}:\t P-value: {data.loc[row, col]:10f}\n"
    return output

In [ ]:
print(significant_pval(C_pvalues_df, "Control"))

In [ ]:
print(significant_pval(E_pvalues_df, "Experimental"))

# Mann-Whitney U Tests 
Testing significance of differences between groups

In [ ]:
# Mann-Whitney U Tests for PlayerScore -- H1 : E > C
score_u_stat, score_pval = stats.mannwhitneyu(np.array(E_data['Task', 'PlayerScore']), 
                                     np.array(C_data['Task', 'PlayerScore']), 
                                     method='exact', alternative='greater')
print(f"The p-value for the distribution of score between groups given H1 : E > C is\t{score_pval}")

In [ ]:
# Mann-Whitney U Tests for Page Submit -- H1 : E > C
submit_u_stat, submit_pval = stats.mannwhitneyu(np.array(E_data['Task', 'Page Submit']), 
                                     np.array(C_data['Task', 'Page Submit']), 
                                     method='exact', alternative='greater')
print(f"The p-value for the distribution of quitting time between groups given H1 : E > C is\t{submit_pval}")

# **Body state distribution and relationships**

Plot the distribution for all body state information across groups (should resemble eachother)

In [ ]:
def plot_bodystate(data, subplots, fig_size):
    v, h = subplots[:2]
    fig, axes = plt.subplots(v, h, figsize=fig_size)
    label_size = 9
    i = 0

    if v * h == 1:
        axes = [axes]  # Convert single-axis to a list
    else:
        axes = axes.flatten()

    data2 = data.droplevel(level=0, axis=1)
    for state in data['Questions'].columns:
        sns.countplot(data= data2, x= state, hue= "Group", ax=axes[i], order= data2[state].unique())
        axes[i].set_title(f"{state} Plot", fontsize=label_size)
        axes[i].set_xlabel(state, fontsize=label_size)
        axes[i].set_ylabel("Density", fontsize=label_size)
        axes[i].tick_params(axis="both", labelsize=label_size)
        axes[i].legend(fontsize= label_size)
        i += 1

    if len(subplots) == 3:
        extra_axes = v * h - i
        for j in range(extra_axes):
            plt.delaxes(axes[i + j])
            
    plt.tight_layout()

In [ ]:
a = data['Questions', 'Sleep'].unique()
a.sort()
a

In [ ]:
plot_bodystate(data, [2, 2], (12, 9))

# **Demographic Information**

In [ ]:
# Gender
labelsize = 9
fig, ax = plt.subplots()
ax.pie(demographic_info["Q_Gender"].value_counts(), labels= demographic_info["Q_Gender"].value_counts().index, startangle=180, autopct='%1.1f%%', textprops= {"fontsize": 10})
ax.set_title("Participant Gender", fontsize= labelsize)

In [ ]:
# Age
labelsize = 9
fig, ax = plt.subplots()
ax.pie(demographic_info["Q_Age"].value_counts(), labels= demographic_info["Q_Age"].value_counts().index, startangle=180, autopct='%1.1f%%', textprops= {"fontsize": 10})
ax.set_title("Participant Age", fontsize= labelsize)

In [ ]:
# Ethnicity
labelsize = 9
fig, ax = plt.subplots()
ax.pie(demographic_info["Q_Ethn"].value_counts(), labels= demographic_info["Q_Ethn"].value_counts().index, startangle=180, autopct='%1.1f%%', textprops= {"fontsize": 10})
ax.set_title("Participant Ethnicity", fontsize= labelsize)

In [ ]:
# Income
labelsize = 9
fig, ax = plt.subplots()
ax.pie(demographic_info["Q_Income"].value_counts(), labels= demographic_info["Q_Income"].value_counts().index, startangle=180, autopct='%1.1f%%', textprops= {"fontsize": 10})
ax.set_title("Participant Income", fontsize= labelsize)